In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import random
import gspread_pandas #pip install gspread-pandas
from gspread_pandas import Spread

df = pd.read_excel("task_data.xlsx")

#clearing erroneous data, getting rid of duplicate keywords in one area and some data with type mismatch and strange column 'good (1)'
df.dropna(inplace=True)
df.drop('good (1)', axis=1, inplace=True)
df = df.drop_duplicates(["area", "keyword"])
df = df[~((df['count'] == '-') | (df['count'] == 'N\A') | (df['y'] == '0x414fe002'))]

#creating a  tableu color dictionary for each area-cluster pair
cols = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
area = df.area.unique()
cluster = df.cluster.unique()
area_cluster = list(itertools.product(area, cluster))
color = []
random.seed(1)
for i in range(len(area)):
    color.extend(random.sample(cols, 4))
area_cluster_color = dict(zip(area_cluster, color))

#creating a new column 'color' with a color value from the dictionary
df['color'] = df.apply(lambda x: area_cluster_color[(x["area"], x["cluster"])], axis=1)

#explicitly specifying column types before sorting and applying filters
types = ["string", int, "string", "string", int, np.float64, np.float64, "string"]
retype = dict(zip(df.columns, types))
df = df.astype(retype)

#sort by area, cluster, cluster_name in ascending order and count in descending order
df = df.sort_values([ 'area', 'cluster', 'cluster_name', 'count' ], ascending=[True, True, True, False])

#connecting to google api and creating a Spread object to interact with the google-spreadsheet
#conf_dir - my local dir, creds.json - my google_secret.json in the local dir which allows me to access the service account, 
#which I previously allowed in the google table
#spread is the url for my google-spreadsheet
conf = gspread_pandas.conf.get_config(conf_dir='C:\\Users\\oanov\\Тестовое', file_name='creds.json')
cred = gspread_pandas.conf.get_creds(config=conf)
spread = Spread(spread="https://docs.google.com/spreadsheets/d/1TuaKIhiv14f3W1OninFqu_-NB5uwSaJeE8j1wdraEws/edit#gid=0", config=conf, creds=cred)

#sending dataframe to google-spreadsheet with pinned header and added filter
spread.df_to_sheet(df, index=False, freeze_headers=True, add_filter=True) # spread.clear_sheet()